# 2021-05-04 Daily Practice

- [x] Practice
- [ ] Learn
- [x] Write
- [ ] Build

---

## Write

Continued the Contact Section (sci-fi) story I started yesterday morning. Started out saying I'd write one page, ended up writing two.

---

## Learn, Review, and Practice

Spent all of the daytime hours on researching, learning, and reviewing for a series of virtual "on-site" Software Engineering interviews I have later this week.

---

## SQL Practice

Spent the evening prepping and practicing for a data engineering takehome test I have tomorrow morning.

- Programming
- Data engineering
- System design
- SQL
- Written communication

### Like Tracker

On [Interview Query](https://www.interviewquery.com/questions/like-tracker):

| events     | table    |
| ---------- | -------- |
| column     | type     |
| user_id    | int      |
| created_at | datetime |
| action     | string   |
| platform   | string   |

The events table tracks every time a user performs a certain action (like, post_enter, etc.) on a platform.

Write a query to determine how many different users gave a like on June 6, 2020.

Output:

| column              | type |
| ------------------- | ---- |
| num_users_gave_like | int  |

```sql
select
    count(distinct user_id) as num_users_gave_like
from events
where created_at = '2020-06-06'
    and action = 'like'
```

### Upsell Transactions

> On [Interview Query](https://www.interviewquery.com/questions/upsell-transactions)

`transactions` table

| column     | type     |
| ---------- | -------- |
| id         | integer  |
| user_id    | integer  |
| created_at | datetime |
| product_id | integer  |
| quantity   | integer  |

We're given a table of product purchases. Each row in the table represents an individual user product purchase.

Write a query to get the number of customers that were upsold by purchasing additional products.

Note that if the customer purchased two things on the same day that does not count as an upsell as they were purchased within a similar timeframe.

Output:

| column                  | type    |
| ----------------------- | ------- |
| num_of_upsold_customers | integer |

My solution:

Got the right answer, but one of the test cases isn't working for some reason.

```sql
with groupings as (
    select
        user_id
        , month(created_at) as month
        , date(created_at) as date
    from transactions
    group by 2, 3
    having count(user_id) > 1
)

select count(distinct user_id) as num_of_upsold_customers
from groupings
```

### Download Facts


> On [Interview Query](https://www.interviewquery.com/questions/download-facts)

user_dimension table

| column     | type |
| ---------- | ---- |
| user_id    | int  |
| account_id | int  |

account_dimension table

| column          | type    |
| --------------- | ------- |
| account_id      | int     |
| paying_customer | boolean |

download_facts table

| column    | type |
| --------- | ---- |
| date      | date |
| user_id   | int  |
| downloads | int  |

Given three tables: user_dimension, account_dimension, and download_facts, find the average number of downloads for free vs paying customers broken out by day.

Note: The account_dimension table maps users to multiple accounts where they could be a paying customer or not. Also, round average_downloads to 2 decimal places.

Output:

| column            | type    |
| ----------------- | ------- |
| date              | date    |
| paying_customer   | boolean |
| average_downloads | float   |

First pass:

```sql
with user_accounts as (
    select
        max(u.account_id) as account_id
        , u.user_id
        , max(paying_customer) as paying_customer
    from user_dimension u
    join account_dimension a
        on u.account_id = a.account_id
    group by 2
)

select
    date
    , paying_customer
    , round(avg(downloads), 2) as average_downloads
from user_accounts u
join download_facts d
    on u.user_id = d.user_id
group by date, paying_customer
order by 1, 2
```

Second pass, with help from the solution:

```sql
select
    date
    , paying_customer
    , round(sum(downloads)/count(distinct u.user_id), 2) as average_downloads
from user_dimension u
inner join account_dimension a
    on u.account_id = a.account_id
left join download_facts d
    on d.user_id = u.user_id
group by 1, 2
```
